In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')
from scipy import spatial
import os
from numpy import array



In [2]:
def cosine_similarity(arr1,arr2):
    ans=1- spatial.distance.cosine(arr1,arr2)
    if(np.isnan(ans)):
        return 0
    else:
        return ans

In [3]:
def get_recommendations(skills,domains):
        df=pd.read_csv("./data/domain_userprofile.csv",index_col='Respondent')
        matches=dict()
        skills=skills.split(",")
        skills=list(map(lambda x:x.lower(),skills))                
        input_path="./data/"
        userdomain=pd.DataFrame(columns=df.columns)
        dictionary=dict()
        domains=domains.split(",")    
        for domain in domains:
            dictionary[domain]=1.0
        userdomain=userdomain.append(dictionary,ignore_index=True)


        df=pd.read_csv(input_path+"languages_profileuser.csv",index_col='Respondent')
        userlanguages=pd.DataFrame(columns=df.columns)
        dictionary=dict()
        for skill in skills:
            if(skill in df.columns):
                dictionary[skill]=1.0
        userlanguages=userlanguages.append(dictionary,ignore_index=True)
        df=pd.read_csv(input_path+"frameworks_profileuser.csv",index_col='Respondent')
        userframeworks=pd.DataFrame(columns=df.columns)
        dictionary=dict()
        for skill in skills:
            if(skill in df.columns):
                dictionary[skill]=1.0
        userframeworks=userframeworks.append(dictionary,ignore_index=True)
        df=pd.read_csv(input_path+"platforms_profileuser.csv",index_col='Respondent')
        userplatforms=pd.DataFrame(columns=df.columns)                
        dictionary=dict()
        for skill in skills:
            if(skill in df.columns):
                dictionary[skill]=1.0
        userplatforms=userplatforms.append(dictionary,ignore_index=True)

        df=pd.read_csv(input_path+"databases_profileuser.csv",index_col='Respondent')
        userdatabases=pd.DataFrame(columns=df.columns)               
        dictionary=dict()
        for skill in skills:
            if(skill in df.columns):
                dictionary[skill]=1.0
        userdatabases=userdatabases.append(dictionary,ignore_index=True)
            #print(userdomain)
        userdomain=np.asarray(userdomain.iloc[0,:].fillna(0))
        userlanguages=np.asarray(userlanguages.iloc[0,:].fillna(0))
        userframeworks=np.asarray(userframeworks.iloc[0,:].fillna(0))
        userplatforms=np.asarray(userplatforms.iloc[0,:].fillna(0))
        userdatabases=np.asarray(userdatabases.iloc[0,:].fillna(0))
                
        jobdomain=pd.read_csv(input_path+"domain_jobprofile.csv",index_col='uniq_id')
        joblanguages=pd.read_csv(input_path+'languages_profilejob.csv',index_col='uniq_id')
        jobframeworks=pd.read_csv(input_path+'frameworks_profilejob.csv',index_col='uniq_id')
        jobplatforms=pd.read_csv(input_path+'platforms_profilejob.csv',index_col='uniq_id')
        jobdatabases=pd.read_csv(input_path+'databases_profilejob.csv',index_col='uniq_id')
        #print(len(jobdomain.index),len(joblanguages.index))
        for i in jobdomain.index:
            #print(i)
            try:
                domain=jobdomain.loc[i,:].fillna(0)
                language=joblanguages.loc[i,:].fillna(0)
                framework=jobframeworks.loc[i,:].fillna(0)
                platform=jobplatforms.loc[i,:].fillna(0)
                database=jobdatabases.loc[i,:].fillna(0)
                job_id=str(i)
                domain=np.asarray(domain)
                language=np.asarray(language)
                framework=np.asarray(framework)
                platform=np.asarray(platform)
                database=np.asarray(database)
                
                score=(0.6*cosine_similarity(domain,userdomain))+(0.4*(cosine_similarity(language,userlanguages)+cosine_similarity(framework,userframeworks)+cosine_similarity(platform,userplatforms)+cosine_similarity(database,userdatabases)))
                matches[job_id]=score
            except KeyError:
                pass
        matches=sorted(matches.items(),key=lambda x:x[1],reverse=True)
        maxscore=matches[0][1]
        threshval=0.3
        threshscore=threshval*maxscore
        print("Threshold Value = "+str(threshval))
        print("Threshold Score = "+str(threshscore))
        truepositive={v for i,v in matches if v>threshscore and v>0.0}
        falsepositive={v for i,v in matches if v<threshscore and v>0.0}
        truenegative={v for i,v in matches if v==0.0}
        tp=len(truepositive)
        fp=len(falsepositive)
        tn=len(truenegative)
        tot=len(matches)
        precision=tp/(tp+fp)
        f1score=2*(precision)/(precision+1)
        print("Precision = "+str(precision))
        print("F1 Score = "+str(f1score))
                
            
        
        recommendations=matches[:10]
        df2=pd.read_csv("./data/dice_com-job_us_sample.csv")
        rows=pd.DataFrame(columns=df2.columns)
        count=0
        for i in recommendations:
            row=df2[df2['uniq_id']==i[0]]
            rows=rows.append(row.iloc[0])
            count=count+1
            #print(row)
        return rows
            

In [5]:

dfuser=pd.read_csv("./data/collaborative/userskills.csv")
dfuser = dfuser.loc[:,~dfuser.columns.duplicated()]
dfuser=dfuser.fillna(0)
dfuser.shape
dfuser.head()
dfuser=dfuser.dropna(thresh=2)
dfuser.shape
m=(np.asscalar(np.int32(max(dfuser["Respondent"]))))
temp=[0]*m
vector=np.array(temp)
count=1
d=dict()
for row in dfuser.iterrows():
        index,data=row
        l=list()
        s=np.asscalar(np.int32(data.values[0]))
        d[s]=np.array(list(data.values[1:]))
        

 
 
dfcont=pd.read_csv("./data/collaborative/recommendations.csv")
sim=list()

for i in range(200):
        l=list()
        l=[0]*200
        sim.append(l)


for key, value in d.items():
        if(key<200):
            
            b=(np.linalg.norm(value))
            for key2,value2 in d.items():
                if(key2<200):
                    
                    a=np.dot(d[key],d[key2])
                    ans=a/(np.linalg.norm(value2)*b)
                    sim[key][key2]=ans

In [6]:
import flask
from flask import *  
app = flask.Flask(__name__, template_folder='templates')  
 
@app.route('/',methods=["GET","POST"])  
def choice ():  
    if flask.request.method == 'GET':
        return render_template('choosetype.html')
    if flask.request.method == 'POST':
        uc=flask.request.form.get('selected_metric')
        if(uc=='content-based filtering'):
            return redirect(url_for("contentbased"))
        else:
            return redirect(url_for("collab"))

@app.route('/contentbased',methods=['GET','POST'])       
def contentbased():
    if flask.request.method == 'GET':
        return(flask.render_template('contbased.html'))
            
    if flask.request.method == 'POST':
    
        #.list()
        d=flask.request.form['domain']
        s=flask.request.form['skills']
        
        df=pd.read_csv("./data/domain_userprofile.csv",index_col='Respondent')
        
        re=get_recommendations(s,d)
        re=re.drop(['joblocation_address','postdate','site_name','shift','uniq_id'],axis=1)
        re=re.rename(columns={"advertiserurl":"advertisement_url","skills":"skills needed","jobid":"job id","employmenttype_jobstatus":"type of employment","jobdescription":"job description","jobtitle":"job title"})
        re=re.reset_index(drop=True)
        re=re.reindex(columns=["company","job title","job description","skills needed","type of employment","advertisement_url","job id"])
        
        return (flask.render_template('positive_j.html', tables=[re.to_html(classes='re')], titles=re.columns.values)) 

@app.route('/collab',methods=['GET','POST'])       
def collab():
    if flask.request.method == 'GET':
        return(flask.render_template('collabfilt.html'))
            
    if flask.request.method == 'POST':
    
        
        user=flask.request.form['userno']
        user=int(user)
        m1=max(sim[user][:user])
        m2=max(sim[user][(user+1):])
        ma=max(m1,m2)
        suser=sim[user].index(ma)

        re=dfcont[dfcont['Respondent']==suser]
        re=re.drop(['joblocation_address','postdate','site_name','shift','uniq_id'],axis=1)
        re=re.rename(columns={"advertiserurl":"advertisement_url","Respondent":"User ID","skills":"skills needed","jobid":"job ID","employmenttype_jobstatus":"type of employment","jobdescription":"job description","jobtitle":"job title"})
        re=re.reset_index(drop=True)
        re=re.reindex(columns=["User ID","company","job title","job description","skills needed","type of employment","advertisement_url","job ID"])

        return (flask.render_template('positive_collab.html', tables=[re.to_html(classes='re')], titles=re.columns.values,u=user,s=suser)) 
if __name__ == '__main__':  
    app.run(debug = False)         

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jun/2021 20:54:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 20:54:43] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Jun/2021 20:54:46] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [14/Jun/2021 20:54:46] "GET /contentbased HTTP/1.1" 200 -


Threshold Value = 0.3
Threshold Score = 0.45413484604514076
Precision = 0.37643835616438354
F1 Score = 0.5469745222929936


127.0.0.1 - - [14/Jun/2021 20:56:01] "POST /contentbased HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 20:57:12] "GET /contentbased HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 20:57:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 20:57:17] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [14/Jun/2021 20:57:17] "GET /collab HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 20:57:24] "POST /collab HTTP/1.1" 200 -
